In [1]:
# Basic code below is from the Get-Tweets.py and from the Async8.2 coding

# Important libraries; and login information
import tweepy
import json
import sys
import nltk

nltk.download('punkt')
nltk.download('stopwords')
from twitter_login_fn import oauth_login
from twitter_login_fn import appauth_login
from DB_fn import save_to_DB
from DB_fn import load_from_DB
from operator import itemgetter
from DB_fn import load_from_DB


import warnings
warnings.filterwarnings('ignore')

# Define arguments for use later in the program
arg1 = '#books' # Looking for tweets with a specific hashtag
arg2 = 4000 # Looking for 4000 tweets
arg3 = 'books_project' # Naming the database
arg4 = 'book_tweets' # Naming the collection

# Setting a search function 
def twitter_search(api, query, max_results=20):
    search_results = [status for status in tweepy.Cursor(api.search, q=query).items(max_results)]
    tweetsvoice = [tweet._json for tweet in search_results]
    return tweetsvoice

# Setting up command line arguments
if __name__ == '__main__':
    args = sys.argv[1:]
    query = arg1
    num_tweets = int(arg2)
    DBname = arg3
    DBcollection = arg4

# Oauth Login
    api = appauth_login()
    print ("Twitter Authorization: ", api)
    
# Accessing tweet results
    result_tweets = twitter_search(api, query, max_results=num_tweets)
    print ('Number of result tweets: ', len(result_tweets))

# Saving results into a database
    DBname = DBname.lower()
    DBname = DBname.replace('#', '')
    DBname = DBname.replace(' ', '')
    DBcollection = DBcollection.lower()
    DBcollection = DBcollection.replace('#', '')
    DBcollection = DBcollection.replace(' ', '')
    
# Saving/loading database
    save_to_DB(DBname, DBcollection, result_tweets)
    load_from_DB(DBname, DBcollection)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lkube\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lkube\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Twitter Authorization:  <tweepy.api.API object at 0x00000270C3459748>
Number of result tweets:  4000
Saved 4000 documents to DB collection books_project book_tweets


In [2]:
# Importing library and setting up to view data in MongoDB
import pymongo
client = pymongo.MongoClient('localhost', 27017)
client.list_database_names()


['TwitterStream',
 'admin',
 'bball',
 'books_0530',
 'books_project',
 'config',
 'disney',
 'local',
 'mylib',
 'people_DB',
 'peopledb',
 'ppl_database',
 'team_blake',
 'team_kelly',
 'team_legend',
 'team_nick',
 'the_voice',
 'usgs']

In [3]:
# Viewing database; and available collection
db = client.books_project
db.list_collection_names()



['book_tweets']

In [4]:
# Naming collection
coll = db.book_tweets
docs = coll.find()

# Converting the document cursor to a list of documents
doclist = list(docs)

In [5]:
# This is how the tweets look now
print(doclist[:1])


[{'_id': ObjectId('5edd8249dc2ca9b657c8fdf4'), 'created_at': 'Mon Jun 08 00:08:50 +0000 2020', 'id': 1269783407233576960, 'id_str': '1269783407233576960', 'text': "RT @InderjitkaurALS: Hi #WritingCommunity, say #hello to all our lovely #writers. Let's do a #writerslift. Leave your links, and promise to…", 'truncated': False, 'entities': {'hashtags': [{'text': 'WritingCommunity', 'indices': [24, 41]}, {'text': 'hello', 'indices': [47, 53]}, {'text': 'writers', 'indices': [72, 80]}, {'text': 'writerslift', 'indices': [93, 105]}], 'symbols': [], 'user_mentions': [{'screen_name': 'InderjitkaurALS', 'name': 'Inderjit Kaur', 'id': 1904467544, 'id_str': '1904467544', 'indices': [3, 19]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_re

In [6]:
msglist = [doc['text'] for doc in doclist if 'text' in doc.keys()]
len(msglist)


4000

In [7]:
all_tokens = [tok for text in msglist for tok in nltk.word_tokenize(text)]
len(all_tokens)


108784

In [8]:
all_tokens[:50]


['RT',
 '@',
 'InderjitkaurALS',
 ':',
 'Hi',
 '#',
 'WritingCommunity',
 ',',
 'say',
 '#',
 'hello',
 'to',
 'all',
 'our',
 'lovely',
 '#',
 'writers',
 '.',
 'Let',
 "'s",
 'do',
 'a',
 '#',
 'writerslift',
 '.',
 'Leave',
 'your',
 'links',
 ',',
 'and',
 'promise',
 'to…',
 'RT',
 '@',
 'noveliciouss',
 ':',
 'We',
 'live',
 'for',
 '#',
 'books',
 '.',
 '—',
 'Umberto',
 'Eco',
 '#',
 'writing',
 '#',
 'amwriting',
 '#']

In [9]:
textFD = nltk.FreqDist(all_tokens)
textFD.most_common(30)


[('#', 12180),
 (':', 6015),
 ('@', 3653),
 ('RT', 2645),
 ('https', 2643),
 (',', 2041),
 ('.', 1872),
 ('the', 1247),
 ('!', 1155),
 ('of', 1120),
 ('books', 1103),
 ('to', 1087),
 ('a', 1043),
 ('and', 972),
 ('for', 735),
 ('I', 689),
 ('The', 642),
 ('in', 613),
 ('on', 508),
 ('Books', 499),
 (';', 467),
 ('book', 461),
 ('is', 457),
 ('?', 450),
 ('you', 449),
 ('by', 416),
 ('...', 399),
 ('&', 368),
 ('’', 361),
 ("'s", 359)]

In [10]:
all_tokens = [tok.lower() for text in msglist for
tok in nltk.word_tokenize(text)]

all_tokens[:30]


['rt',
 '@',
 'inderjitkaurals',
 ':',
 'hi',
 '#',
 'writingcommunity',
 ',',
 'say',
 '#',
 'hello',
 'to',
 'all',
 'our',
 'lovely',
 '#',
 'writers',
 '.',
 'let',
 "'s",
 'do',
 'a',
 '#',
 'writerslift',
 '.',
 'leave',
 'your',
 'links',
 ',',
 'and']

In [11]:
nltk_stopwords = nltk.corpus.stopwords.words('english')
len(nltk_stopwords)


179

In [12]:
nltk_stopwords


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [13]:
import re
def alpha_filter(w):
    pattern = re.compile('^[^a-z]+$')
    if (pattern.match(w)):
        return True
    else:
        return False


In [14]:
token_list = [tok for tok in all_tokens if not alpha_filter(tok)]
token_list[:30]


['rt',
 'inderjitkaurals',
 'hi',
 'writingcommunity',
 'say',
 'hello',
 'to',
 'all',
 'our',
 'lovely',
 'writers',
 'let',
 "'s",
 'do',
 'a',
 'writerslift',
 'leave',
 'your',
 'links',
 'and',
 'promise',
 'to…',
 'rt',
 'noveliciouss',
 'we',
 'live',
 'for',
 'books',
 'umberto',
 'eco']

In [15]:
# Looking at retweet counts; displaying the information and comparing that information
def FilterByGreaterThanRetweet(tweets, nNumberOfRetweets ):
    lTweets = []
    for tweet in tweets:
        if tweet['retweet_count'] > nNumberOfRetweets:
            lTweets.append(tweet)
    return lTweets
def FilterByLessThanRetweet(tweets, nNumberOfRetweets ):
    lTweets = []
    for tweet in tweets:
        if tweet['retweet_count'] < nNumberOfRetweets:
            lTweets.append(tweet)
    return lTweets
lTweetsLessThanTen        = FilterByLessThanRetweet(doclist, 50)
lTweetsGreaterThanHundred = FilterByGreaterThanRetweet(doclist, 100)

print("Number of tweets that have less than 50  retweets:",len(lTweetsLessThanTen))
print("Number of tweets that have more than 100 retweets:",len(lTweetsGreaterThanHundred))

Number of tweets that have less than 50  retweets: 3445
Number of tweets that have more than 100 retweets: 311


In [17]:
# Display some of the tweets with info about date, user, message, and number of retweets.
def print_tweet_data(tweets):
    for tweet in tweets:
        print('\nDate:', tweet['created_at'])
        print('From:', tweet['user']['name'])
        print('Message', tweet['text'])
        print('Number of Retweets:',tweet['retweet_count'])
        
# Display some of the tweets with info about date, user, message, and number of retweets.
print_tweet_data(lTweetsLessThanTen[:10])


Date: Mon Jun 08 00:08:50 +0000 2020
From: Jellybeans Closet 4U
Message RT @InderjitkaurALS: Hi #WritingCommunity, say #hello to all our lovely #writers. Let's do a #writerslift. Leave your links, and promise to…
Number of Retweets: 8

Date: Mon Jun 08 00:08:49 +0000 2020
From: Edgar Alvarado13 e.v.🌞
Message RT @noveliciouss: We live for #books.
— Umberto Eco
#writing #amwriting #reading #bookslover #love #books
#Art Zirine https://t.co/ng2kkWey…
Number of Retweets: 19

Date: Mon Jun 08 00:08:46 +0000 2020
From: CircleOfBooks
Message RT @CircleofBooks: #scifi #kindle #iartg #asmsg #books
Amidst Alien Stars
https://t.co/QW9ESAhGBf 
@CGrahamSciFi 
#tbr #asmsg #iartg #amrea…
Number of Retweets: 9

Date: Mon Jun 08 00:08:44 +0000 2020
From: Literazee
Message Reading The Catcher in the Rye in a social way https://t.co/GDEQ3CCx8c Not just #books, #comics and #graphicnovels… https://t.co/nAhb4GJKDn
Number of Retweets: 0

Date: Mon Jun 08 00:07:48 +0000 2020
From: WHBrown
Message #writing #am

In [18]:
# Display some of the tweets with info about date, user, message, and number of retweets.
print_tweet_data(lTweetsGreaterThanHundred[:10])


Date: Mon Jun 08 00:08:11 +0000 2020
From: Jen
Message RT @StevenLenton: Celebrating over a million book sales (how did that happen?!) with a lovely giveaway! Simply follow, retweet and be in th…
Number of Retweets: 1006

Date: Mon Jun 08 00:04:35 +0000 2020
From: Valery Sobolev
Message RT @judehaste_write: "An Adrenalin Rush"
#StayPositive_StaySafe 
Let the books do the walking #escapism 
🌈USA &amp; UK 🌈 https://t.co/78iawh5K3…
Number of Retweets: 536

Date: Mon Jun 08 00:04:24 +0000 2020
From: Valery Sobolev
Message RT @judehaste_write: #worldbookday2020 thanks to all the #authors #artists who support and share our works. https://t.co/7ewHDOHE2P  
🔆 #Re…
Number of Retweets: 399

Date: Mon Jun 08 00:03:16 +0000 2020
From: L. G. Cullens
Message RT @LGCullens: #EcoFiction  #Adventure  #Nature #Books
https://t.co/VGLD7eCrwi https://t.co/nHdg6kwHjV
Number of Retweets: 205

Date: Mon Jun 08 00:02:51 +0000 2020
From: KzS Seebär
Message RT @anglrsg9: 'I totally loved this book; it is refres

In [19]:
# Code from class to get hashtag, mentions, urls frequencies

def get_entities(tweet):
    if 'entities' in tweet.keys():
        mentions = [user_mention['screen_name'] for user_mention in tweet['entities']['user_mentions']]
        hashtags = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]    
        urls = [urlitem['url'] for urlitem in tweet['entities']['urls']]    
        urls = urls + [urlitem['expanded_url'] for urlitem in tweet['entities']['urls']]
        return mentions, hashtags, urls
    else:
        return [], [], []

# Use a main so can get command line arguments
if __name__ == '__main__':
    args = sys.argv[1:]
    if not args or len(args) < 3:
        print('usage: python twitter_hashtags.py <DBname> <DBcollection> <number>')
        #sys.exit(1) #'d this so that could be run in jupyter notebook'
    DBname = 'books_project'
    DBcollection = 'book_tweets'
    limit = int(20)

    tweet_results = load_from_DB(DBname, DBcollection)

# Create a frequency dictionary of the hashtags
hashtag_fd = {}
for tweet in tweet_results:
    (mentions, hashtags, urls) = get_entities(tweet)
    for tag in hashtags:
        if not tag in hashtag_fd:
            hashtag_fd[tag] = 1
        else:
            hashtag_fd[tag] += 1

# Sorting the dictionary by frequency values
hashtags_sorted = sorted(hashtag_fd.items(), key=itemgetter(1), reverse=True)

# print out the top number of words with frequencies
# go through the first 20 tweets and find the entities
print("Top", limit, "Frequency Hashtags")
for (word, frequency) in hashtags_sorted[:limit]:
        print (word, frequency)

usage: python twitter_hashtags.py <DBname> <DBcollection> <number>
Top 20 Frequency Hashtags
books 955
Books 433
DataScience 286
BigData 280
Analytics 260
IoT 225
IIoT 224
Python 186
Kindle 171
IARTG 157
PyTorch 129
RStats 124
Amazon 120
reading 117
book 109
WritingCommunity 100
writerslift 91
amreading 80
NLProc 80
ebook 72
